#          --------------                 **La prédiction des prix de vol**  --------------------------------------

In [1]:
#importing basics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df_train = pd.read_excel('Data_Train.xlsx')
df_test = pd.read_excel('Test_set.xlsx')

In [3]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


*Combinons les deux données pour pouvoir faire en bloc les traitement*

In [50]:
final_df = df_train.append(df_test)

In [18]:
# Fonction de description de la fréquence des variables object
def frequences(x):
    for i in x.columns:
        if (x[i].dtype == 'object'):
            print(x[i].value_counts())
            print('----')

*Séparation en date , mois et année*

In [51]:
final_df['Date'] = final_df['Date_of_Journey'].str.split('/').str[0]
final_df['Month'] = final_df['Date_of_Journey'].str.split('/').str[1]
final_df['Year'] = final_df['Date_of_Journey'].str.split('/').str[2]

*Convertion en entier*

In [52]:
final_df['Date'] = final_df['Date'].astype(int)
final_df['Month'] = final_df['Month'].astype(int)
final_df['Year'] = final_df['Year'].astype(int)

In [53]:
final_df.drop('Date_of_Journey', axis= 1, inplace = True)

In [54]:
final_df['Year'].unique()

array([2019])

*Vue l'ensemble de la donnée est enregistré en 2019 on va le supprimé*

In [55]:
final_df.drop('Year', axis=1, inplace=True)

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13354 entries, 0 to 2670
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          13354 non-null  object 
 1   Source           13354 non-null  object 
 2   Destination      13354 non-null  object 
 3   Route            13353 non-null  object 
 4   Dep_Time         13354 non-null  object 
 5   Arrival_Time     13354 non-null  object 
 6   Duration         13354 non-null  object 
 7   Total_Stops      13353 non-null  object 
 8   Additional_Info  13354 non-null  object 
 9   Price            10683 non-null  float64
 10  Date             13354 non-null  int32  
 11  Month            13354 non-null  int32  
dtypes: float64(1), int32(2), object(9)
memory usage: 1.2+ MB


In [12]:
final_df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,24,3
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,1,5


### Décomposons le temps d'arrivé en heure et minute et en supprimant la variable initale

In [56]:
final_df['Arrival_Time'] = final_df['Arrival_Time'].apply(lambda x: x.split(' ')[0])
final_df['Arrival_hour'] = final_df['Arrival_Time'].apply(lambda x: x.split(':')[0])
final_df['Arrival_min'] = final_df['Arrival_Time'].apply(lambda x: x.split(':')[1])

final_df.drop('Arrival_Time', axis =1, inplace =True)

final_df['Arrival_hour'] = final_df['Arrival_hour'].astype(int)
final_df['Arrival_min'] = final_df['Arrival_min'].astype(int)

###  Faisons le même traitement pour les heures de depart

In [57]:
final_df['Dep_hour'] = final_df['Dep_Time'].apply(lambda x: x.split(':')[0])
final_df['Dep_min'] = final_df['Dep_Time'].apply(lambda x: x.split(':')[1])

final_df.drop('Dep_Time', axis =1, inplace =True)

final_df['Dep_hour'] = final_df['Dep_hour'].astype(int)
final_df['Dep_min'] = final_df['Dep_min'].astype(int)

In [58]:
final_df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897.0,24,3,1,10,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662.0,1,5,13,15,5,50


In [59]:
final_df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [60]:
final_df['duration_hour'] = final_df['Duration'].str.split(' ').str[0].str.split('h').str[0]final_df['Total_Stops'] = final_df['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, 'nan' : 5, '4 stops':4})


In [61]:
final_df['duration_hour'].unique()

array(['2', '7', '19', '5', '4', '15', '21', '25', '13', '12', '26', '22',
       '23', '20', '10', '6', '11', '8', '16', '3', '27', '1', '14', '9',
       '18', '17', '24', '30', '28', '29', '37', '34', '38', '35', '36',
       '47', '33', '32', '31', '42', '39', '5m', '41', '40'], dtype=object)

On remarque q'il y a un '5m' dans le lot. Vue qu'aucun vol ne peut durer 5 min on va devoir le supprimer

In [62]:
final_df[final_df['duration_hour']== '5m']

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_hour
6474,Air India,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,5m,2.0,No info,17327.0,6,3,16,55,16,50,5m
2660,Air India,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,5m,2.0,No info,NaN,12,3,16,55,16,50,5m


In [63]:
final_df.drop(6474,axis=0,inplace=True)
final_df.drop(2660,axis=0,inplace=True)

In [64]:
# Conversion en entier e
final_df['duration_hour'] = final_df['duration_hour'].astype('int')

Récuperation des minutes et mettons toute la période en minutes

In [65]:
final_df['duration_min'] = final_df['Duration'].str.split(' ').str[1].str.split('m').str[0]
final_df['duration_min'] = final_df['duration_min'].fillna(0)
final_df['duration_min'] = final_df['duration_min'].astype('int')

In [70]:
final_df['duration_tot_min'] = (final_df['duration_hour'] * 60) + final_df['duration_min']

In [71]:
final_df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_hour,duration_min,duration_tot_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0.0,No info,3897.0,24,3,1,10,22,20,2,50,170
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2.0,No info,7662.0,1,5,13,15,5,50,7,25,445
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2.0,No info,13882.0,9,6,4,25,9,25,19,0,1140
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1.0,No info,6218.0,12,5,23,30,18,5,5,25,325
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1.0,No info,13302.0,1,3,21,35,16,50,4,45,285


In [72]:
final_df.drop(['Duration', 'duration_hour', 'duration_min', 'Route'],axis=1,inplace=True)

In [73]:
final_df.head(2)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_tot_min
0,IndiGo,Banglore,New Delhi,0.0,No info,3897.0,24,3,1,10,22,20,170
1,Air India,Kolkata,Banglore,2.0,No info,7662.0,1,5,13,15,5,50,445


In [74]:
frequences(final_df)

Jet Airways                          4745
IndiGo                               2564
Air India                            2190
Multiple carriers                    1543
SpiceJet                             1026
Vistara                               608
Air Asia                              405
GoAir                                 240
Multiple carriers Premium economy      16
Jet Airways Business                    8
Vistara Premium economy                 5
Trujet                                  1
Name: Airline, dtype: int64
----
Delhi       5681
Kolkata     3581
Banglore    2752
Mumbai       881
Chennai      456
Name: Source, dtype: int64
----
Cochin       5681
Banglore     3581
Delhi        1582
New Delhi    1170
Hyderabad     881
Kolkata       456
Name: Destination, dtype: int64
----
No info                         10490
In-flight meal not included      2426
No check-in baggage included      396
1 Long layover                     20
Change airports                     8
Business cl

In [75]:
# Faisons une transformation catégorielle
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()

final_df['Airline'] = lbe.fit_transform(final_df['Airline'])
final_df['Source'] = lbe.fit_transform(final_df['Source'])
final_df['Destination'] = lbe.fit_transform(final_df['Destination'])
final_df['Additional_Info'] = lbe.fit_transform(final_df['Additional_Info'])

In [77]:
print(final_df.shape)
final_df.head(2)

(13351, 13)


,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Arrival_hour,Arrival_min,Dep_hour,Dep_min,duration_tot_min
0,3,0,5,0.0,8,3897.0,24,3,1,10,22,20,170
1,1,3,0,2.0,8,7662.0,1,5,13,15,5,50,445


In [88]:
df_test = final_df[final_df['Price'].isnull()] # pour selection que les Purchase qui n'ont pas de valeur
df_test.drop('Price', axis=1, inplace=True)
df_train = final_df[~final_df['Price'].isnull()] # le signe ~ permet de sectionner le contraire de la condition

C:\Users\LENOVO\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [93]:
df_train.isnull().sum()
df_train = df_train.fillna(1)

##### *Enregistrement des données préparées*

In [118]:
df_train.to_csv('train_dataset.csv', index=False)
df_test.to_csv('test_dataset.csv', index= False)

### Je vais encoder la variable

In [99]:
hot_train_df = pd.get_dummies(df_train, columns = ['Airline'])

In [115]:
hot_train_df.columns

Index(['Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Price',
       'Date', 'Month', 'Arrival_hour', 'Arrival_min', 'Dep_hour', 'Dep_min',
       'duration_tot_min', 'Airline_0', 'Airline_1', 'Airline_2', 'Airline_3',
       'Airline_4', 'Airline_5', 'Airline_6', 'Airline_7', 'Airline_8',
       'Airline_9', 'Airline_10', 'Airline_11'],
      dtype='object')